In [12]:
from seldon_deploy_sdk import Configuration, ApiClient, PredictApi, SeldonDeploymentsApi, ModelMetadataServiceApi, DriftDetectorApi, BatchJobsApi, BatchJobDefinition
from seldon_deploy_sdk.auth import OIDCAuthenticator
from seldon_deploy_sdk.rest import ApiException
import pprint
import json

In [2]:
SD_IP = "34.74.242.235"
config = Configuration()
config.host = f"http://{SD_IP}/seldon-deploy/api/v1alpha1"
config.oidc_client_id = "sd-api"
config.oidc_server = f"http://{SD_IP}/auth/realms/deploy-realm"
config.oidc_client_secret = "sd-api-secret"
config.auth_method = 'client_credentials'
config.verify_ssl = False
auth = OIDCAuthenticator(config)
config.id_token = auth.authenticate()
seldon_api_client = ApiClient(configuration=config, authenticator=auth)

In [9]:
MODEL_NAME = "fraud-detection"

DEPLOYMENT_NAME = f"{MODEL_NAME}"
MODEL_LOCATION = f"gs://tom-seldon-examples/fraud-workshop/models/josh-pretrained/"

NAMESPACE = "seldon-demos"

PREPACKAGED_SERVER = "XGBOOST_SERVER"

CPU_REQUESTS = "0.1"
MEMORY_REQUESTS = "1Gi"

CPU_LIMITS = "0.1"
MEMORY_LIMITS = "1Gi"

In [10]:
mldeployment = {
    "kind": "SeldonDeployment",
    "metadata": {
        "name": DEPLOYMENT_NAME,
        "namespace": NAMESPACE,
        "labels": {
            "fluentd": "true"
        }
    },
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "spec": {
        "name": DEPLOYMENT_NAME,
        "annotations": {
            "seldon.io/engine-seldon-log-messages-externally": "true"
        },
        "protocol": "seldon",
        "transport": "rest",
        "predictors": [
            {
                "componentSpecs": [
                    {
                        "spec": {
                            "containers": [
                                {
                                    "name": f"{DEPLOYMENT_NAME}-container",
                                    "resources": {
                                        "requests": {
                                            "cpu": CPU_REQUESTS,
                                            "memory": MEMORY_REQUESTS
                                        },
                                        "limits": {
                                            "cpu": CPU_LIMITS,
                                            "memory": MEMORY_LIMITS
                                        }
                                    }
                                }
                            ]
                        }
                    }
                ],
                "name": "default",
                "replicas": 1,
                "traffic": 100,
                "graph": {
                    "implementation": PREPACKAGED_SERVER,
                    "modelUri": MODEL_LOCATION,
                    "name": f"{DEPLOYMENT_NAME}-container",
                    "endpoint": {
                        "type": "REST"
                    },
                    "parameters": [],
                    "children": [],
                    "logger": {
                        "mode": "all"
                    }
                }
            }
        ]
    },
    "status": {}
}

In [11]:
deployment_api = SeldonDeploymentsApi(seldon_api_client)
deployment_api.create_seldon_deployment(namespace="seldon-demos", mldeployment=mldeployment)

{'api_version': 'machinelearning.seldon.io/v1alpha2',
 'kind': 'SeldonDeployment',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': None,
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'labels': {'fluentd': 'true'},
              'managed_fields': None,
              'name': 'fraud-detection',
              'namespace': 'seldon-demos',
              'owner_references': None,
              'resource_version': None,
              'self_link': None,
              'uid': None},
 'spec': {'annotations': {'seldon.io/engine-seldon-log-messages-externally': 'true'},
          'name': 'fraud-detection',
          'oauth_key': None,
          'oauth_secret': None,
          'predictors': [{'annotations': None,
                          'component_specs': [{'hpa_spec': None,


## Check Status of Deployment

In [ ]:
deployment_status = ''
while deployment_status != "Available":
    try:
        api_response = deployment_api.read_seldon_deployment(DEPLOYMENT_NAME, NAMESPACE)
        pprint.pprint(api_response.status.state.replace('\'', ''))
        deployment_status = api_response.status.state
    except ApiException as e:
        print("Exception when calling SeldonDeploymentsApi->read_seldon_deployment: %s\n" % e)

In [ ]:
api_response = deployment_api.read_seldon_deployment(DEPLOYMENT_NAME, NAMESPACE)
pprint.pprint(api_response)

## Send a Prediction

### Not Fraud

In [16]:
# create an instance of the API class
predict_api = PredictApi(seldon_api_client)
NOT_FRAUD_PREDICTION = {
    "data": {
        "names": ["step", "type", "amount", "oldBalanceOrig", "newBalanceOrig",
                  "oldBalanceDest", "newBalanceDest", "errorBalanceOrig", "errorBalanceDest"],
        "ndarray": [
            [205, 1, 63243.44, -1.00, -1.00, 1853683.32, 1916926.76, 63243.44, 0]
        ]
    }
}

try:
    api_response = predict_api.predict_seldon_deployment(DEPLOYMENT_NAME, NAMESPACE, NOT_FRAUD_PREDICTION)
    print(api_response)
except ApiException as e:
    print("Exception when calling PredictApi->predict_seldon_deployment: %s\n" % e)

{'data': {'names': [], 'ndarray': [0.0003859958960674703]}, 'meta': {'requestPath': {'fraud-detection-container': 'seldonio/xgboostserver:1.13.1'}}}


### Fraud

In [17]:
# create an instance of the API class
predict_api = PredictApi(seldon_api_client)
FRAUD_PREDICTION = {
    "data": {
        "names": ["step", "type", "amount", "oldBalanceOrig", "newBalanceOrig",
                  "oldBalanceDest", "newBalanceDest", "errorBalanceOrig", "errorBalanceDest"],
        "ndarray": [
            [629, 1, 2433009.28, 2433009.28, 0.00, 0.00, 2433009.28, 0.00, 0.00]
        ]
    }
}

try:
    api_response = predict_api.predict_seldon_deployment(DEPLOYMENT_NAME, NAMESPACE, FRAUD_PREDICTION)
    print(api_response)
except ApiException as e:
    print("Exception when calling PredictApi->predict_seldon_deployment: %s\n" % e)

{'data': {'names': [], 'ndarray': [0.9998043179512024]}, 'meta': {'requestPath': {'fraud-detection-container': 'seldonio/xgboostserver:1.13.1'}}}


## Get list of Deployments Via Namespace

In [ ]:
deployment_api = SeldonDeploymentsApi(seldon_api_client)

try:
    api_response = deployment_api.list_seldon_deployments(NAMESPACE)
    pprint.pprint(api_response)
except ApiException as e:
    print("Exception when calling SeldonDeploymentsApi->list_seldon_deployments: %s\n" % e)

In [ ]:
{
    "data": {
        "names": ["step", "type", "amount", "oldBalanceOrig", "newBalanceOrig",
                  "oldBalanceDest", "newBalanceDest", "errorBalanceOrig", "errorBalanceDest"],
        "ndarray": [
            [629, 1, 2433009.28, 2433009.28, 0.00, 0.00, 2433009.28, 0.00, 0.00],
            [205, 1, 63243.44, -1.00, -1.00, 1853683.32, 1916926.76, 63243.44, 0]
        ]
    }
}